In [4]:
# !pip install vaderSentiment
# !pip install imblearn
# !pip install xgboost

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
# Health & Personal Care Category
# df = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz', sep='\t', error_bad_lines=False)

# Watches Category
df = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz', sep='\t', error_bad_lines=False)
df

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960199,US,39508379,REH8UQZAXQS40,B00005OT9I,2153632,Xhilaration&#174; Silver Dial Bangle Watch,Watches,2,17,17,N,N,Great prize - but size??,It's a great watch - but you don't have a chan...,2001-12-13
960200,US,44704251,RJ14QPZEOI9P8,B00005QEM6,903687397,Armitron Men's Titanium 3 Eye Sport Analog Chr...,Watches,5,9,10,N,N,A very classy watch!,"Well worth the price. This is a light weight, ...",2001-11-08
960201,US,40571775,R2B1G5650WWFCE,B00005QEME,252821780,Men's Timex Internet Messenger Sport Watch,Watches,5,3,16,N,N,This is a great watch,"Dear Targeteers,<BR>This watch is exelent. it ...",2001-11-06
960202,US,44474855,R2MMGPUWXXOFI2,B00004YK0H,118389241,Energizer 393 Button Cell Battery,Watches,4,0,0,N,N,Now watt a minute here.,"In the old days, the common hearing battery in...",2001-04-05


In [13]:
df = df[df.star_rating != 3]

In [19]:
df['review_class'] = ((df['star_rating'] == 4) | (df['star_rating'] == 5)).astype(int)

In [20]:
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_class
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31,1
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31,1
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31,0
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31,1
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960199,US,39508379,REH8UQZAXQS40,B00005OT9I,2153632,Xhilaration&#174; Silver Dial Bangle Watch,Watches,2,17,17,N,N,Great prize - but size??,It's a great watch - but you don't have a chan...,2001-12-13,0
960200,US,44704251,RJ14QPZEOI9P8,B00005QEM6,903687397,Armitron Men's Titanium 3 Eye Sport Analog Chr...,Watches,5,9,10,N,N,A very classy watch!,"Well worth the price. This is a light weight, ...",2001-11-08,1
960201,US,40571775,R2B1G5650WWFCE,B00005QEME,252821780,Men's Timex Internet Messenger Sport Watch,Watches,5,3,16,N,N,This is a great watch,"Dear Targeteers,<BR>This watch is exelent. it ...",2001-11-06,1
960202,US,44474855,R2MMGPUWXXOFI2,B00004YK0H,118389241,Energizer 393 Button Cell Battery,Watches,4,0,0,N,N,Now watt a minute here.,"In the old days, the common hearing battery in...",2001-04-05,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960204 entries, 0 to 960203
Data columns (total 15 columns):
marketplace          960204 non-null object
customer_id          960204 non-null int64
review_id            960204 non-null object
product_id           960204 non-null object
product_parent       960204 non-null int64
product_title        960202 non-null object
product_category     960204 non-null object
star_rating          960204 non-null int64
helpful_votes        960204 non-null int64
total_votes          960204 non-null int64
vine                 960204 non-null object
verified_purchase    960204 non-null object
review_headline      960197 non-null object
review_body          960056 non-null object
review_date          960200 non-null object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [23]:
analyzer = SentimentIntensityAnalyzer()

In [24]:
# Return dictionary of VADER sentiment scores for given sentence. Keys: 'neg', 'neu', 'pos', 'compound'
def sentiment_analyzer_scores(sentence, verbose=False):
    score = analyzer.polarity_scores(sentence)
    if verbose: print("{}".format(str(score)))
    
    return score
    
def sentiment_analyzer_scores_neg(sentence):
    return sentiment_analyzer_scores(sentence)['neg']

def sentiment_analyzer_scores_neu(sentence):
    return sentiment_analyzer_scores(sentence)['neu']

def sentiment_analyzer_scores_pos(sentence):
    return sentiment_analyzer_scores(sentence)['pos']

def sentiment_analyzer_scores_compound(sentence):
    return sentiment_analyzer_scores(sentence)['compound']

In [25]:
df_small = df.sample(n=20000)

# Deal with missing values for any of the columns we will be using
# df_small['star_rating'] = df_small['star_rating'].astype(int)
df_small['review_headline'] = df_small['review_headline'].fillna('.')
df_small['review_body'] = df_small['review_body'].fillna('.')
df_small['review_fulltext'] = df_small['review_headline'] + '. ' + df_small['review_body']

df_small['neg'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_neg)
df_small['neu'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_neu)
df_small['pos'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_pos)
df_small['compound'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_compound)

df_small.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 925507 to 342078
Data columns (total 21 columns):
marketplace          20000 non-null object
customer_id          20000 non-null int64
review_id            20000 non-null object
product_id           20000 non-null object
product_parent       20000 non-null int64
product_title        20000 non-null object
product_category     20000 non-null object
star_rating          20000 non-null int64
helpful_votes        20000 non-null int64
total_votes          20000 non-null int64
vine                 20000 non-null object
verified_purchase    20000 non-null object
review_headline      20000 non-null object
review_body          20000 non-null object
review_date          20000 non-null object
review_class         20000 non-null int64
review_fulltext      20000 non-null object
neg                  20000 non-null float64
neu                  20000 non-null float64
pos                  20000 non-null float64
compound             20000 n

In [26]:
df_small['star_rating'].value_counts()

5    12953
4     3925
1     1931
2     1191
Name: star_rating, dtype: int64

In [27]:
df_numer = df_small.select_dtypes(include='number')
df_numer.info()
df_numer

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 925507 to 342078
Data columns (total 10 columns):
customer_id       20000 non-null int64
product_parent    20000 non-null int64
star_rating       20000 non-null int64
helpful_votes     20000 non-null int64
total_votes       20000 non-null int64
review_class      20000 non-null int64
neg               20000 non-null float64
neu               20000 non-null float64
pos               20000 non-null float64
compound          20000 non-null float64
dtypes: float64(4), int64(6)
memory usage: 1.7 MB


,customer_id,product_parent,star_rating,helpful_votes,total_votes,review_class,neg,neu,pos,compound
925507,32300045,41385950,4,0,1,1,0.000,1.000,0.000,0.0000
183942,44094333,744159218,2,0,1,0,0.262,0.738,0.000,-0.9431
487818,2309326,364754751,5,0,0,1,0.000,1.000,0.000,0.0000
35599,10801539,445678999,5,0,0,1,0.000,0.717,0.283,0.8581
364874,48649983,793248827,4,0,0,1,0.000,0.580,0.420,0.4404
...,...,...,...,...,...,...,...,...,...,...
912778,34085305,876715961,5,2,2,1,0.170,0.530,0.300,0.6900
564794,47018436,952380499,5,2,2,1,0.000,0.625,0.375,0.9324
640785,20597251,961043582,5,0,0,1,0.000,0.644,0.356,0.9612
742544,41586788,740675803,5,0,0,1,0.000,0.679,0.321,0.9112


In [28]:
word_tokenizer = RegexpTokenizer(r'\w+')
df_small['review_wordlist'] = df_small['review_fulltext'].map(lambda x: word_tokenizer.tokenize(x.lower()))

In [29]:
df_small

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,...,review_headline,review_body,review_date,review_class,review_fulltext,neg,neu,pos,compound,review_wordlist
925507,US,32300045,R2F0865SHKCZE,B000SQP7XG,41385950,Timex Men's T2M467 T Series Chronograph Black ...,Watches,4,0,1,...,timex t series watch,"product arrived quickly, as described, and muc...",2009-01-11,1,"timex t series watch. product arrived quickly,...",0.000,1.000,0.000,0.0000,"[timex, t, series, watch, product, arrived, qu..."
183942,US,44094333,R3COYL0TV95B79,B003OQ4M92,744159218,Timex Women's Q7B779 Calfskin 8mm Black Replac...,Watches,2,0,1,...,... a five star rating but I don't know how go...,I should give it a five star rating but I don'...,2015-03-07,0,... a five star rating but I don't know how go...,0.262,0.738,0.000,-0.9431,"[a, five, star, rating, but, i, don, t, know, ..."
487818,US,2309326,RRWQWCQS6LQYN,B00134L90C,364754751,Casio Men's MTP3050D-2AV Classic 10-Year Batte...,Watches,5,0,0,...,Five Stars,excelente,2014-05-01,1,Five Stars. excelente,0.000,1.000,0.000,0.0000,"[five, stars, excelente]"
35599,US,10801539,R1XUFV7JL7S9F3,B005FUF320,445678999,red line Men's RL-18101-02 Compressor Collecti...,Watches,5,0,0,...,too happy.,"Very good n nice watch, a friend of mine took...",2015-07-28,1,"too happy.. Very good n nice watch, a friend ...",0.000,0.717,0.283,0.8581,"[too, happy, very, good, n, nice, watch, a, fr..."
364874,US,48649983,R58CAM47P5KDD,B002UG1B0U,793248827,Energizer 393 Silver Oxide 10 Batteries (SR754W),Watches,4,0,0,...,Four Stars,better than Duracell,2014-10-04,1,Four Stars. better than Duracell,0.000,0.580,0.420,0.4404,"[four, stars, better, than, duracell]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912778,US,34085305,RFYD473IMB81D,B001S7VKGC,876715961,Freestyle Unisex FS81263 Shark Classic Green P...,Watches,5,2,2,...,Great watch,I use this watch when I go Surfing and Skimboa...,2009-09-25,1,Great watch. I use this watch when I go Surfin...,0.170,0.530,0.300,0.6900,"[great, watch, i, use, this, watch, when, i, g..."
564794,US,47018436,R1K3QLXKXGXRDC,B008PJCRVS,952380499,Seiko Women's SNDX95 Sportura Classic Ceramic ...,Watches,5,2,2,...,Perfect and Stunning,This was a christmas present for my wife. She ...,2014-01-01,1,Perfect and Stunning. This was a christmas pre...,0.000,0.625,0.375,0.9324,"[perfect, and, stunning, this, was, a, christm..."
640785,US,20597251,R3W5BSXC6DUHLW,B00791R1MI,961043582,Casio Men's Solar Sport Combination Watch,Watches,5,0,0,...,Solar power cool watch.,"Got it 3 days after put order, Love amazon.<br...",2013-08-07,1,Solar power cool watch.. Got it 3 days after p...,0.000,0.644,0.356,0.9612,"[solar, power, cool, watch, got, it, 3, days, ..."
742544,US,41586788,RH01MDDXKFADP,B001A67HDG,740675803,Casio G-shock GLX5600-1 G-Lide Red Watch Mens,Watches,5,0,0,...,Love the watch...the red is amazing!,"I have many watches, as I never leave the hous...",2013-01-15,1,Love the watch...the red is amazing!. I have m...,0.000,0.679,0.321,0.9112,"[love, the, watch, the, red, is, amazing, i, h..."


In [30]:
df_small['review_wordlist'] = df_small['review_wordlist'].map(lambda x: ' '.join(x))

In [35]:
X = df_small.loc[:, ['helpful_votes', 'total_votes', 'neg', 'neu', 'pos', 'compound', 'review_wordlist']]
y = df_small.loc[:, 'review_class']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [33]:
# create our training data list - this is a list of strings, with each string being a review

clean_train_data = []

for train_data in X_train['review_wordlist']:
    clean_train_data.append(train_data)

In [149]:
# create test data list

clean_test_data = []

for test_data in X_test['review_wordlist']:
    clean_test_data.append(test_data)

In [150]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our 
# features are single words

vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             max_features = 5000,
                             ngram_range = (1, 3))

In [151]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

# convert to array

train_data_features = train_data_features.toarray()

In [152]:
# check shapes

train_data_features.shape, test_data_features.shape

((15000, 5000), (5000, 5000))

In [156]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()
np.random.choice(vocab, 200)

array(['making', 'sized wrist', 'complications', 'really wanted',
       'watch white', 'operation', 'nice classy', 'looks works', '15 20',
       'penny', 'dial easy read', 'read', 'watch got', 'great features',
       'metallic', 'left', 'read watch', 'course', 'goes', 'original box',
       'minutes', 'arrive', 'watch set', 'keeps accurate time',
       'buy great', 'alert', 'watch need', 'switch', 'ocean',
       'unfortunately', 'buy new watch', 'watch lots', 'possible', '000',
       'uncomfortable', 'br br quality', 'automatics', 'watch nice',
       'ebay', 'como', 'exceptionally', 'stuck', 'watch problem',
       'casual', 'explained', 'hacking', 'nice looking', 'chance',
       'watch built', 'sealed', 'hole', 'contrast', 'sits', 'larger size',
       'watch design', 'reaction', 'fraction', 'continue', 'look really',
       'wearing watch', 'girlfriend loves', 'quality excellent',
       'nice touch', 'watch use', 'price little', 'nice br',
       'watch gorgeous', 'mother pe

In [37]:
y_train.value_counts()

1    12654
0     2346
Name: review_class, dtype: int64

In [41]:
smote = SMOTE()
X_train_numeric = X_train.drop(columns='review_wordlist')
X_test_numeric = X_test.drop(columns='review_wordlist')

X_train_resampled, y_train_resampled = smote.fit_sample(X_train_numeric, y_train) 
y_train_resampled.value_counts()

1    12654
0    12654
Name: review_class, dtype: int64

In [42]:
# Test out different classification algorithms
# Logistic Regression

logreg = LogisticRegression(fit_intercept=True, C=1e12, solver='liblinear', penalty='l2')
logreg.fit(X_train_resampled, y_train_resampled)

LogisticRegression(C=1000000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
y_hat_test_lr = logreg.predict(X_test_numeric)
y_hat_train_lr = logreg.predict(X_train_resampled)

In [44]:
# We could subtract the two columns. If values or equal, difference will be zero. Then count number of zeros 
residuals = np.abs(y_train_resampled - y_hat_train_lr)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    21488
1     3820
Name: review_class, dtype: int64
0    0.84906
1    0.15094
Name: review_class, dtype: float64


In [45]:
residuals = np.abs(y_test - y_hat_test_lr)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    4283
1     717
Name: review_class, dtype: int64
0    0.8566
1    0.1434
Name: review_class, dtype: float64


In [46]:
print(classification_report(y_train_resampled, y_hat_train_lr))

# Confusion Matrix for training set - Logistic Regression
cf_lr = confusion_matrix(y_train_resampled, y_hat_train_lr)
cf_lr

              precision    recall  f1-score   support

           0       0.85      0.85      0.85     12654
           1       0.85      0.85      0.85     12654

    accuracy                           0.85     25308
   macro avg       0.85      0.85      0.85     25308
weighted avg       0.85      0.85      0.85     25308



array([[10726,  1928],
       [ 1892, 10762]])

In [47]:
print(classification_report(y_test, y_hat_test_lr))

# Confusion Matrix for training set - Logistic Regression
cm_lr = confusion_matrix(y_test, y_hat_test_lr)
cm_lr

              precision    recall  f1-score   support

           0       0.52      0.84      0.64       776
           1       0.97      0.86      0.91      4224

    accuracy                           0.86      5000
   macro avg       0.75      0.85      0.78      5000
weighted avg       0.90      0.86      0.87      5000



array([[ 650,  126],
       [ 591, 3633]])

In [50]:
plot_confusion_matrix(logreg, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    # disp.ax_.set_title(title)

#     print(title)
#     print(disp.confusion_matrix)

NameError: name 'class_names' is not defined

In [51]:
# fit logistic regression model

lr = LogisticRegression(penalty = 'l2')
lr.fit(train_data_features, y_train)

NameError: name 'train_data_features' is not defined

In [52]:
lr.score(train_data_features, y_train)

NameError: name 'train_data_features' is not defined

In [53]:
lr.score(test_data_features, y_test)

NameError: name 'test_data_features' is not defined

In [54]:
# K Nearest Neighbors

# Instantiate KNeighborsClassifier
clf_knn = KNeighborsClassifier()

# Fit the classifier
clf_knn.fit(X_train_resampled, y_train_resampled)

# Predict on the training set
y_hat_knn_train = clf_knn.predict(X_train_resampled)

print(classification_report(y_train_resampled, y_hat_knn_train))

cm_knn_train = confusion_matrix(y_train_resampled, y_hat_knn_train)
cm_knn_train

              precision    recall  f1-score   support

           0       0.89      0.92      0.90     12654
           1       0.92      0.88      0.90     12654

    accuracy                           0.90     25308
   macro avg       0.90      0.90      0.90     25308
weighted avg       0.90      0.90      0.90     25308



array([[11672,   982],
       [ 1497, 11157]])

In [55]:
y_hat_knn_test = clf_knn.predict(X_test_numeric)

print(classification_report(y_test, y_hat_knn_test))

cm_knn_test = confusion_matrix(y_test, y_hat_knn_test)
cm_knn_test

              precision    recall  f1-score   support

           0       0.49      0.74      0.59       776
           1       0.95      0.86      0.90      4224

    accuracy                           0.84      5000
   macro avg       0.72      0.80      0.74      5000
weighted avg       0.88      0.84      0.85      5000



array([[ 571,  205],
       [ 594, 3630]])

In [56]:
# Improve model performance by finding the best value for K

In [57]:
clf_knn2 = KNeighborsClassifier()

# Fit the classifier
clf_knn2.fit(train_data_features, y_train)

# Predict on the training set
y_hat_knn2 = clf_knn2.predict(train_data_features)

print(classification_report(y_train, y_hat_knn2))

cm_knn2 = confusion_matrix(y_train, y_hat_knn2)
cm_knn2

NameError: name 'train_data_features' is not defined

In [58]:
# Decision Trees

clf_dt = DecisionTreeClassifier(criterion='entropy')

clf_dt.fit(X_train_resampled, y_train_resampled)

# Predict on the training set
y_hat_dt_train = clf_dt.predict(X_train_resampled)

cm_dt_train = confusion_matrix(y_train_resampled, y_hat_dt_train)
cm_dt_train

array([[12639,    15],
       [  551, 12103]])

In [59]:
print(classification_report(y_train_resampled, y_hat_dt_train))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     12654
           1       1.00      0.96      0.98     12654

    accuracy                           0.98     25308
   macro avg       0.98      0.98      0.98     25308
weighted avg       0.98      0.98      0.98     25308



In [60]:
y_hat_dt_test = clf_dt.predict(X_test_numeric)

print(classification_report(y_test, y_hat_dt_test))

cm_dt_test = confusion_matrix(y_test, y_hat_dt_test)
cm_dt_test

              precision    recall  f1-score   support

           0       0.45      0.71      0.55       776
           1       0.94      0.84      0.89      4224

    accuracy                           0.82      5000
   macro avg       0.70      0.78      0.72      5000
weighted avg       0.86      0.82      0.84      5000



array([[ 550,  226],
       [ 670, 3554]])

In [61]:
clf_dt2 = DecisionTreeClassifier(criterion='entropy')

# Fit the classifier
clf_dt2.fit(train_data_features, y_train)

# Predict on the training set
y_hat_dt2 = clf_dt2.predict(train_data_features)

print(classification_report(y_train, y_hat_dt2))

cm_dt2 = confusion_matrix(y_train, y_hat_dt2)
cm_dt2

NameError: name 'train_data_features' is not defined

In [62]:
# Bagged Trees

# Instantiate a BaggingClassifier
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_leaf_nodes=100), 
                                 n_estimators=20)

# Fit to the training data
bagged_tree.fit(X_train_resampled, y_train_resampled)

bagged_tree.score(X_train_resampled, y_train_resampled)

0.8852536747273589

In [63]:
bagged_tree.score(X_test_numeric, y_test)

0.8372

In [64]:
y_hat_bagged_train = bagged_tree.predict(X_train_resampled)

cm_bagged_train = confusion_matrix(y_train_resampled, y_hat_bagged_train)
cm_bagged_train

array([[11701,   953],
       [ 1951, 10703]])

In [65]:
print(classification_report(y_train_resampled, y_hat_bagged_train))

              precision    recall  f1-score   support

           0       0.86      0.92      0.89     12654
           1       0.92      0.85      0.88     12654

    accuracy                           0.89     25308
   macro avg       0.89      0.89      0.89     25308
weighted avg       0.89      0.89      0.89     25308



In [66]:
y_hat_bagged_test = bagged_tree.predict(X_test_numeric)

print(classification_report(y_test, y_hat_bagged_test))

cm_bagged_test = confusion_matrix(y_test, y_hat_bagged_test)
cm_bagged_test

              precision    recall  f1-score   support

           0       0.49      0.84      0.61       776
           1       0.97      0.84      0.90      4224

    accuracy                           0.84      5000
   macro avg       0.73      0.84      0.76      5000
weighted avg       0.89      0.84      0.85      5000



array([[ 648,  128],
       [ 686, 3538]])

In [67]:
# Random Forest

In [68]:
# Instantiate and fit a RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, max_depth = 10)
forest.fit(X_train_resampled, y_train_resampled)
forest.score(X_train_resampled, y_train_resampled)

0.8996759917812549

In [69]:
y_hat_forest = forest.predict(X_train_resampled)

cm_forest = confusion_matrix(y_train_resampled, y_hat_forest)
cm_forest

array([[11872,   782],
       [ 1757, 10897]])

In [70]:
print(classification_report(y_train_resampled, y_hat_forest))

              precision    recall  f1-score   support

           0       0.87      0.94      0.90     12654
           1       0.93      0.86      0.90     12654

    accuracy                           0.90     25308
   macro avg       0.90      0.90      0.90     25308
weighted avg       0.90      0.90      0.90     25308



In [71]:
y_hat_forest_test = forest.predict(X_test_numeric)

print(classification_report(y_test, y_hat_forest_test))

cm_forest_test = confusion_matrix(y_test, y_hat_forest_test)
cm_forest_test

              precision    recall  f1-score   support

           0       0.50      0.84      0.62       776
           1       0.97      0.84      0.90      4224

    accuracy                           0.84      5000
   macro avg       0.73      0.84      0.76      5000
weighted avg       0.89      0.84      0.86      5000



array([[ 649,  127],
       [ 662, 3562]])

In [117]:
# Grid Search / Random Search to find optimal parameters

In [118]:
# SVM / Linear SVC

In [96]:
# clf_svc = SVC(kernel='linear')
# clf_svc.fit(X_train_resampled, y_train_resampled)

# # Predict on the training set
# y_hat_svc = clf_svc.predict(X_train_resampled)

# print(classification_report(y_train_resampled, y_hat_svc))

# cm_svc = confusion_matrix(y_train_resampled, y_hat_svc)
# cm_svc


In [121]:
# Ensemble Classifiers

In [134]:
# Adaboost
# Instantiate an AdaBoostClassifier
clf_ab = AdaBoostClassifier()

clf_ab.fit(X_train_resampled, y_train_resampled)
clf_ab.score(X_train_resampled, y_train_resampled)

0.4098626435487503

In [135]:
y_hat_ab_train = clf_ab.predict(X_train_resampled)

cm_ab_train = confusion_matrix(y_train_resampled, y_hat_ab_train)
cm_ab_train

array([[5028, 2195, 1166,  433,   60],
       [3495, 2743, 1547,  877,  220],
       [1725, 1957, 2154, 1951, 1095],
       [ 541,  795, 1472, 2809, 3265],
       [ 239,  534,  738, 1903, 5468]])

In [136]:
print(classification_report(y_train_resampled, y_hat_ab_train))

              precision    recall  f1-score   support

           1       0.46      0.57      0.51      8882
           2       0.33      0.31      0.32      8882
           3       0.30      0.24      0.27      8882
           4       0.35      0.32      0.33      8882
           5       0.54      0.62      0.58      8882

    accuracy                           0.41     44410
   macro avg       0.40      0.41      0.40     44410
weighted avg       0.40      0.41      0.40     44410



In [137]:
y_hat_ab_test = clf_ab.predict(X_test_numeric)

print(classification_report(y_test, y_hat_ab_test))

cm_ab_test = confusion_matrix(y_test, y_hat_ab_test)
cm_ab_test

              precision    recall  f1-score   support

           1       0.43      0.52      0.47       434
           2       0.16      0.31      0.21       261
           3       0.16      0.23      0.19       435
           4       0.24      0.29      0.26       881
           5       0.81      0.62      0.70      2989

    accuracy                           0.50      5000
   macro avg       0.36      0.39      0.37      5000
weighted avg       0.58      0.50      0.53      5000



array([[ 227,  116,   51,   35,    5],
       [  84,   81,   59,   25,   12],
       [  82,   81,  101,   98,   73],
       [  54,   79,  146,  253,  349],
       [  83,  149,  258,  654, 1845]])

In [138]:
# Instantiate an GradientBoostingClassifier
clf_gb = GradientBoostingClassifier()

clf_gb.fit(X_train_resampled, y_train_resampled)
clf_gb.score(X_train_resampled, y_train_resampled)

0.4839450574195001

In [139]:
y_hat_gb_train = clf_gb.predict(X_train_resampled)

cm_gb_train = confusion_matrix(y_train_resampled, y_hat_gb_train)
cm_gb_train

array([[5145, 2113, 1120,  399,  105],
       [2073, 4454, 1361,  753,  241],
       [1237, 1983, 3049, 1713,  900],
       [ 410,  957, 1414, 3197, 2904],
       [ 228,  616,  761, 1630, 5647]])

In [140]:
print(classification_report(y_train_resampled, y_hat_gb_train))

              precision    recall  f1-score   support

           1       0.57      0.58      0.57      8882
           2       0.44      0.50      0.47      8882
           3       0.40      0.34      0.37      8882
           4       0.42      0.36      0.39      8882
           5       0.58      0.64      0.60      8882

    accuracy                           0.48     44410
   macro avg       0.48      0.48      0.48     44410
weighted avg       0.48      0.48      0.48     44410



In [141]:
y_hat_gb_test = clf_gb.predict(X_test_numeric)

print(classification_report(y_test, y_hat_gb_test))

cm_gb_test = confusion_matrix(y_test, y_hat_gb_test)
cm_gb_test

              precision    recall  f1-score   support

           1       0.45      0.51      0.48       434
           2       0.16      0.34      0.22       261
           3       0.17      0.25      0.20       435
           4       0.25      0.27      0.26       881
           5       0.80      0.62      0.70      2989

    accuracy                           0.50      5000
   macro avg       0.36      0.40      0.37      5000
weighted avg       0.58      0.50      0.54      5000



array([[ 223,  118,   54,   29,   10],
       [  76,   89,   62,   23,   11],
       [  74,   94,  108,   75,   84],
       [  41,   84,  166,  237,  353],
       [  84,  181,  257,  601, 1866]])

In [52]:
# Clustering

from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score

num_clusters = np.arange(2, 20)

# Set number of clusters at initialization time
for n in num_clusters:
    k_means = KMeans(n_clusters=n) 

    # Run the clustering algorithm
    k_means.fit(df_numer) 

    # Generate cluster index values for each row
    cluster_assignments = k_means.predict(df_numer)
    
    print(n, calinski_harabasz_score(df_numer, cluster_assignments))

2 59486.25585177998
3 77674.18325588538
4 96384.21852818847
5 114546.82979016086
6 127600.80444243301
7 141010.59498579273
8 150682.09351094894
9 161561.0248727341
10 166964.61084244298
11 172383.19385670617
12 179948.81991115178
13 182327.44955418463
14 185306.0671946048
15 187196.97618489747
16 187075.00888801456
17 186023.28848008186
18 185515.0192828231
19 184560.6592451715
20 182653.62410723022
21 177354.84392849842
22 175636.95905210843
23 172503.3659090606
24 170951.62031425143
25 169277.9078372501
26 167893.9965466513
27 166426.02562539277
28 163768.2050005787
29 164446.90279474566
30 163345.56115803414
31 163941.37842270927
32 161616.26932644393
33 162014.7044419554
34 162781.62265305087
35 163483.25020053884
36 164121.61442848394
37 165910.86684445763
38 166263.34904917312
39 166967.81410185085
40 169879.35195707544
41 170036.8446218407
42 172317.16871043303
43 173225.85042507053
44 176405.45462941405
45 175951.96694708266
46 179420.62087730106
47 181230.39342520965
48 181994

In [53]:
cluster_assignments

array([20, 48, 43, ..., 40, 34, 23], dtype=int32)

In [54]:
k_means.labels_

array([20, 48, 43, ..., 40, 34, 23], dtype=int32)

In [55]:
# This code builds on the previous example

# Note that we could also pass in k_means.labels_ instead of cluster_assignments
print(calinski_harabasz_score(df_numer, cluster_assignments))

180807.4371004135
